In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import neurokit2 as nk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import re
import os

In [2]:
DATA_PATH = '/srv/scratch3/bruxit/bruxit_all_fnorm/'
DATA_PATH = "/Users/nanashuka/Document/Zurich/UZH/master_project/sensor-data-analysis-pipeline/back-end/data/"
original_sampling = 2000
selected_sampling = 1000

In [3]:
def resample_signal(signal, sampling_rate=2000, selected_sampling=1000):
    return nk.signal_resample(signal, sampling_rate=sampling_rate, desired_sampling_rate=selected_sampling, method="pandas")

In [5]:
existing_patients_recordings = {}
for folder in os.listdir(DATA_PATH):
        if not os.path.isdir(DATA_PATH + folder):
            continue
        if not re.search('p(.*?)_', folder):
            continue
        patient_id = re.search('p(.*?)_', folder).group(1)

        patient_week_folder = DATA_PATH + folder

        csv_files = [f for f in os.listdir(patient_week_folder) if f.endswith(".csv")]

        night_id_list= []
        night_id_recorder = {}
        for csv in csv_files:
            night_id = re.search("[0-9]+", csv).group(0)
            night_id_list.append(night_id)

            if "location_Bites" not in csv:
                recorder = re.search('.(?=F)', csv).group(0)
                night_id_recorder[night_id] = recorder

        if night_id_list:
            week = re.search('wk(.*)', folder).group(1)
            night_id_list = list(set(night_id_list))
            night_id_list = sorted(night_id_list)

            for night_id in night_id_list:
                if patient_id in existing_patients_recordings:
                    existing_patients_recordings[patient_id].append({
                        "week": week,
                        "night_id": night_id,
                        "recorder": night_id_recorder[night_id]
                    })
                else:
                    existing_patients_recordings[patient_id] = [{
                        "week": week,
                        "night_id": night_id,
                        "recorder": night_id_recorder[night_id]
                    }]
print(existing_patients_recordings)

{'3': [{'week': '21-22', 'night_id': '0100004', 'recorder': 'a'}, {'week': '5-6', 'night_id': '0809455', 'recorder': 'a'}, {'week': '5-6', 'night_id': '0816565', 'recorder': 'a'}, {'week': '5-6', 'night_id': '0902175', 'recorder': 'a'}, {'week': '3-4', 'night_id': '0602473', 'recorder': 'a'}, {'week': '3-4', 'night_id': '0612150', 'recorder': 'a'}, {'week': '13-14', 'night_id': '0100002', 'recorder': 'a'}], '2': [{'week': '1-2', 'night_id': '0107285', 'recorder': 'e'}, {'week': '1-2', 'night_id': '0123035', 'recorder': 'e'}, {'week': '1-2', 'night_id': '0204451', 'recorder': 'e'}, {'week': '1-2', 'night_id': '0213314', 'recorder': 'e'}, {'week': '5-6', 'night_id': '0521390', 'recorder': 'e'}, {'week': '3-4', 'night_id': '0315230', 'recorder': 'e'}, {'week': '13-14', 'night_id': '1722373', 'recorder': 'd'}], '1': [{'week': '21-22', 'night_id': '2020582', 'recorder': 'c'}, {'week': '1', 'night_id': '0901260', 'recorder': 'c'}, {'week': '1', 'night_id': '1022102', 'recorder': 'c'}, {'week

In [8]:
general_model = xgb.XGBClassifier(n_estimators=100, objective='binary:logistic',
        eval_metric='logloss', subsample=0.6, max_depth=3, learning_rate=0.1, colsample_bytree=1.0)
first = True
for patient_id, recordings in existing_patients_recordings.items():
    print("Patient: ", patient_id)
    patient_model = xgb.XGBClassifier(n_estimators=100, objective='binary:logistic',
        eval_metric='logloss', subsample=0.6, max_depth=3, learning_rate=0.1, colsample_bytree=1.0)
    df = pd.DataFrame({'MR': [], 'ML': [], 'Bites': []})
    for recording in recordings:
        try:
            week = recording["week"]
            night_id = recording["night_id"]
            recorder = recording["recorder"]
            print(f"p{patient_id}, wk{week}, {night_id}{recorder}")
            # Load data
            loc = pd.read_csv(DATA_PATH + "p" + patient_id + "_wk" + week + "/" + night_id + recorder + "location_Bites.csv")
            # data = pd.read_csv(DATA_PATH + "p" + patient_id + "_wk" + week + "/" + night_id + recorder + "Fnorm.csv")
            range_min = 0
            range_max = int(loc.iloc[2,1]) + 1
            dur_len = float(loc.iloc[0,2]) + float(loc.iloc[1,2]) + float(loc.iloc[2,2])
            if dur_len*original_sampling*2 > range_max:
                range_max = int(dur_len*2000*2)
            print('range_max: ', range_max)
            data_itr = pd.read_csv(f'{DATA_PATH}p{patient_id}_wk{week}/{night_id}{recorder}Fnorm.csv', chunksize=range_max, iterator=True, usecols=['MR','ML'])
            data = data_itr.get_chunk()
            print('data.shape: ', data.shape)
            # print('data: ', data.describe())
            
            bites = np.zeros(data.shape[0], dtype=int)
            data = data.dropna()
            # print('data.shape after drop: ', data.shape)
            print(f"label bites, wk{week}, {night_id}{recorder}")
            for i in range(0, data.shape[0]):
                if i < int(float(loc.iloc[0,0])) or (i > int(float(loc.iloc[0,1])) and i < int(float(loc.iloc[1,0]))) or (i > int(float(loc.iloc[1,1])) and i < int(float(loc.iloc[2,0]))) or i > int(float(loc.iloc[2,1])):
                    bites[i] = 0
                else:
                    bites[i] = 1
            bites = bites[data.index]
            MR = data['MR'].values.tolist()
            ML = data['ML'].values.tolist()
            if (len(bites) != len(MR)):
                print('/////////////////////////////////////')
                print('len(bites): ', len(bites))
                print('len(MR): ', len(MR))
                print('/////////////////////////////////////')
            # print('MR: ', MR[:10])
            # print('ML: ', ML[:10])
            MR = resample_signal(signal=MR, sampling_rate=original_sampling, selected_sampling=selected_sampling)
            ML = resample_signal(signal=ML, sampling_rate=original_sampling, selected_sampling=selected_sampling)
            bites = resample_signal(signal=bites, sampling_rate=original_sampling, selected_sampling=selected_sampling)
            # df.loc[:,'Bites'] = bites
            df_p = pd.DataFrame({'MR': MR, 'ML': ML, 'Bites': bites})
            print('df_p.shape: ', df_p.shape)
            print('df.shape: ', df.shape)
            # print('df: ', df_p.describe())
            
            df = df.append(df_p, ignore_index=True)
            print()
        except Exception as e:
            print(e)
            print()
            continue
    
    x = df.iloc[:,:2].copy()
    y = df.iloc[:,-1].copy()
    x = np.array(x.values.tolist())
    y = np.array(y.values.tolist())
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25) # Split data for test and training
    SC = StandardScaler()
    x_train = pd.DataFrame(SC.fit_transform(x_train))
    x_test = pd.DataFrame(SC.transform(x_test))
    print(f"fit model p{patient_id}")
    patient_model.fit(x_train, y_train)
    print(f"save model p{patient_id}")
    patient_model.save_model(DATA_PATH + '/models/' +f"p{patient_id}_model.json")

    print("fit general model")
    if first:
        general_model.fit(x_train, y_train)
        first = False
    else:
        general_model.fit(x_train, y_train, xgb_model=general_model)

general_model.save_model(DATA_PATH + '/models/' + "general_model.json")
print("save general model")

Patient:  3
p3, wk21-22, 0100004a
range_max:  489511
data.shape:  (489511, 2)
data:                    MR            ML
count  489511.000000  4.895110e+05
mean        0.000091 -2.335562e-07
std         3.849291  1.288763e+00
min       -71.793000 -1.757000e+02
25%        -0.302290 -3.872200e-02
50%         0.000733  3.616700e-05
75%         0.306670  3.927800e-02
max       108.550000  8.888100e+01
label bites, wk21-22, 0100004a
df_p.shape:  (244756, 3)
df.shape:  (0, 3)

p3, wk5-6, 0809455a
range_max:  27396
data.shape:  (27396, 2)
data:                   MR            ML
count  27396.000000  27396.000000
mean       0.000019      0.000442
std        0.682977      1.855528
min      -12.705000    -16.463000
25%       -0.085409     -0.327133
50%       -0.003090      0.029702
75%        0.080449      0.539615
max       25.175000     22.501000
label bites, wk5-6, 0809455a


/var/folders/50/2w03ns450yl9ht7z9dkhtr1w0000gn/T/ipykernel_10894/1695330115.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_p, ignore_index=True)


df_p.shape:  (13698, 3)
df.shape:  (244756, 3)

p3, wk5-6, 0816565a
range_max:  20829
data.shape:  (20829, 2)
data:                   MR            ML
count  20829.000000  20829.000000
mean      -0.000232      0.001881
std        1.825176      5.696589
min      -14.500000   -114.570000
25%       -0.612850     -2.403500
50%       -0.017522      0.086490
75%        0.554120      2.405300
max       17.400000    120.110000
label bites, wk5-6, 0816565a


/var/folders/50/2w03ns450yl9ht7z9dkhtr1w0000gn/T/ipykernel_10894/1695330115.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_p, ignore_index=True)


df_p.shape:  (10414, 3)
df.shape:  (258454, 3)

p3, wk5-6, 0902175a
range_max:  18216
data.shape:  (18216, 2)
data:                   MR            ML
count  18216.000000  18216.000000
mean       0.000264     -0.000039
std        3.783902      1.461487
min      -52.385000    -10.225000
25%       -0.705965     -0.550782
50%       -0.013310      0.061732
75%        0.639030      0.815513
max      103.440000      9.179900
label bites, wk5-6, 0902175a


...

df_p.shape:  (19128, 3)
df.shape:  (148428, 3)

p1, wk1, 1222325c
range_max:  47678
data.shape:  (47678, 2)
data:                   MR            ML
count  47678.000000  47678.000000
mean      -0.000047     -0.000013
std        0.122124      1.381749
min       -1.478200    -17.247000
25%       -0.043328     -0.574365
50%        0.002210      0.004575
75%        0.047989      0.650145
max        1.734100      8.678600
label bites, wk1, 1222325c


/var/folders/50/2w03ns450yl9ht7z9dkhtr1w0000gn/T/ipykernel_10894/1695330115.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_p, ignore_index=True)


df_p.shape:  (23839, 3)
df.shape:  (167556, 3)

fit model p1


/var/folders/50/2w03ns450yl9ht7z9dkhtr1w0000gn/T/ipykernel_10894/1695330115.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_p, ignore_index=True)


save model p1
fit general model
save general model
